数据分割补充

    前面见过，可以通过实验测试来对学习器的泛化误差进行评估进而做出选择。
    为此，需要使用一个“测试集”（testing set）来测试学习器对新样本的判断能力，然后以测
    试集上的“测试误差”（testing error）作为泛化误差的近似。
    通常我们假设测试样本也是从样本真实分布中独立同分布采样而得，但需要注意的是，测试集应
    该尽可能与训练集互斥。
    【互斥】即测试样本应尽量不在训练集中出现，未在训练过程中使用。

    测试样本为什么需要尽可能不出现在训练集中呢？为了理解这一点，不妨考虑这样一个场景：
        老师出了10道习题让同学们练习，考试时老师又用同样的这10道题作为测试题，这个考试
        成绩能否有效反映出同学们学得好不好呢？
        答案是否定的，可能有的同学只会做这10道题却能得高分。

    回到我们的问题上来，我们希望得到泛化性能强的模型，好比是希望同学们对课程学的很好，获
    得了对所学知识“举一反三”的能力；训练样本相当于给同学们练习的习题，测试过程则相当于考
    试。显然，若测试样本呢被用作训练了，则得到的将是过于“乐观”的估计结果

    可是，我们只有一个包含了m个样例的数据集D={(x1, y1),(x2,y2)(x3,y3)……(xn, yn)}

    既要训练，又要测试，怎样才能做到呢？
        答案是：通过对D进行适当的处理，从中产生出训练集S和测试集T。（这个也是我们之前一直在做的事情）

    几种常见的做法
        （1）留出法
        （2）交叉验证法
        （3）自助法

1 留出法

    “留出法”（hold-out）直接将数据集D划分为两个互斥的集合，其中一个集合作为训练集S，
    另一个作为测试集T，即D=S∪T，S∩T=Θ。在S上训练出模型后，用T来评估其测试误差，作为泛
    化误差的估计。

    需要注意的是，测试/训练集的划分要尽可能保持数据分布的一致性，避免因数据划分过程引入
    额外的偏差而对最终结果产生影响，例如：在分类任务中至少要保持样本的类别比例相似。

    如果从采样（sampling）的角度来看待数据集的划分过程，则保留类别比例的采样方式通常称为“分层采样”（stratified sampling）
    例如：通过对D进行分层采样而获得含70%样本呢的训练集S和含30%样本的测试集T
         若D包含500个正例，500个反例，则分层抽样得到：
            S应包含350个正例，350个反例
            T则包含150个正例，150个反例
         若S、T中样本类别和比例差别很大，则误差估计将由训练/测试数据分布的差别而产生误差

    另外一个需要注意的问题是，即便在给定训练测试集的样本比例后，仍然存在多种划分方式对初始数据集D进行分别。
    例如：上面的例子中，可以把D中的样本排序，然后把前350个正例放到训练集中，也可以把最
        后350个正例放到训练集中，这些不同的划分将导致不同的训练/测试集，相应的，模型评
        估的结果也会有差别。

    因此，单词使用留出法得到的估计结果往往不够稳定可靠，在使用留出法时，一般要采用若干次
    随机划分、重复进行实验评估后取平均值作为留出法的评估结果。
    例如：进行100次随机划分，每次产生一个训练/测试集可用于实验评估，100次后就得到100个
         结果，而流出法返回的则是这100个结果的平均。

    此外，我们希望评估的使用D训练出的模型的性能，但留出法需划分训练/测试集，这就会导致一个窘境：
        （1）若令训练集S包含绝大多数样本，则训练出的模型可能更接近于用D训练出来的模型
            ，但由于T比较小，评估结果可能不够稳定准确。
        （2）若令测试集T多包含一些样本，则训练集S与D差别更大了，被评估的模型与用D训练
            出的模型相比可能有较大差比，从而降低了评估结果的保真性（fidelity）。

    这个问题没有完美的解决方案，常见的做法是将大约2/3--4/5的样本用于训练，剩余样本用于测试

    # 用python实现留出法
    from sklearn.model_selection import train_test_split
    # 使用train_test_solit划分测试集和训练集
    train_x, test_x, train_y, test_y= train_test_split(
        x, y, test_size=0.2, random_state=0
    )

    在留出法中，有一个特例：留一法（Leave-One-Out,简称LOO），则每次抽取一个样本作为
    测试集。显然，留一法不受随机样本划分方式的影响，因为m个样本只有唯一的方式划分为m个子
    集，每个子集包含个样本：

In [1]:
# 用python实现留一法
from sklearn.model_selection import LeaveOneOut

data = [0,1,2,3,4,5,6,7,8,9]

loo = LeaveOneOut()

for train,test in loo.split(data):
    print(f"{train}\t{test}")

[1 2 3 4 5 6 7 8 9]	[0]
[0 2 3 4 5 6 7 8 9]	[1]
[0 1 3 4 5 6 7 8 9]	[2]
[0 1 2 4 5 6 7 8 9]	[3]
[0 1 2 3 5 6 7 8 9]	[4]
[0 1 2 3 4 6 7 8 9]	[5]
[0 1 2 3 4 5 7 8 9]	[6]
[0 1 2 3 4 5 6 8 9]	[7]
[0 1 2 3 4 5 6 7 9]	[8]
[0 1 2 3 4 5 6 7 8]	[9]


    留一法的优缺点
        （1）优点：留一法使用的训练集与初始训练集相比只少了以恶搞样本，这就是的在绝大多
            数的情况下，留一法中被实际评估的模型与期望评估的用D训练出的模型很相似。因
            此留一法的评估结果往往被认为比较准确。
        （2）缺点：在数据集比较大时，训练m个模型的计算开销可能是难以忍受的（例如数据集
            包含100万个样本，则需训练100万个模型，而这还是在未考虑算法调参的抢矿下。）